In [54]:
print("Bismillah")

Bismillah


# **Installations**

In [55]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u275-b01-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
# Let's import the libraries we will need
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import re

In [ ]:
# # Import and create a new SQLContext 
# from pyspark.sql import SQLContext

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)

# **Pre-Processing**

In [61]:
# Read the country CSV file into an RDD.
country_lines = spark.read.csv('/content/drive/MyDrive/BDA/Assignment02/country-list.csv', sep = ',',  inferSchema = True)
country_lines = country_lines.selectExpr("_c0 as Country", "_c1 as Code")
country_lines.show()
country_lines.printSchema()
type(country_lines)
country_lines.count()

+-------------------+----+
|            Country|Code|
+-------------------+----+
|        Afghanistan| AFG|
|            Albania| ALB|
|            Algeria| ALG|
|     American Samoa| ASA|
|            Andorra| AND|
|             Angola| ANG|
|           Anguilla| AIA|
|Antigua and Barbuda| ATG|
|          Argentina| ARG|
|            Armenia| ARM|
|              Aruba| ARU|
|          Australia| AUS|
|            Austria| AUT|
|         Azerbaijan| AZE|
|            Bahamas| BAH|
|            Bahrain| BHR|
|         Bangladesh| BAN|
|           Barbados| BRB|
|            Belarus| BLR|
|            Belgium| BEL|
+-------------------+----+
only showing top 20 rows

root
 |-- Country: string (nullable = true)
 |-- Code: string (nullable = true)



211

In [62]:
country_lines.take(3)

[Row(Country='Afghanistan', Code=' AFG'),
 Row(Country='Albania', Code=' ALB'),
 Row(Country='Algeria', Code=' ALG')]

In [ ]:
# Convert each line into a pair of words


In [ ]:
# Convert each pair of words into a tuple


In [ ]:
# Create the DataFrame, look at schema and contents
# countryDF = sqlContext.createDataFrame(country_tuples, ["country", "code"])
# countryDF.printSchema()
# countryDF.take(3)


In [63]:
# Read tweets JSON file
users = spark.read.json("/content/drive/MyDrive/BDA/Assignment02/users.json")
users.show()
users.printSchema()

+--------------------+-----------+-------------+--------------------+------------------+---------------------+---------------------+--------------------+--------------------+---------------+
|                 _id|coordinates|retweet_count|              source|          tweet_ID|tweet_followers_count|tweet_mentioned_count|          tweet_text|                user|      user_name|
+--------------------+-----------+-------------+--------------------+------------------+---------------------+---------------------+--------------------+--------------------+---------------+
|[578ffa8e7eb9513f...|       null|            0|<a href="http://t...|755891629932675072|                  461|                    1|RT @ochocinco: I ...|[[2011-12-27T09:0...|        koteras|
|[578ffa8f7eb9513f...|       null|            0|<a href="http://t...|755891632759681024|                 4601|                    3|RT @NiallOfficial...|[[2012-10-19T00:4...|AllieLovesR5_1D|
|[578ffa917eb9513f...|       null|           

In [64]:
users.columns

['_id',
 'coordinates',
 'retweet_count',
 'source',
 'tweet_ID',
 'tweet_followers_count',
 'tweet_mentioned_count',
 'tweet_text',
 'user',
 'user_name']

In [65]:
users.count()

11188

In [66]:
# Clean the data: some tweets are empty. Remove the empty tweets using filter() 
users = users.filter(users.tweet_text.isNotNull())
users.count()

11188

In [53]:
# My mongodb checking query
# users.filter((users.tweet_text.contains("England")) & (~(users.tweet_text.contains("UEFA")))).count()

35

In [67]:
def ascii_ignore(x):
  x = x.encode('ascii', 'ignore').decode('ascii')
  # x = re.sub(r'\W+', ' ', x)
  x = re.sub(r'[^a-zA-Z0-9]', ' ', x)
  # x.replace('[^a-zA-Z]', '')
  return x

ascii_udf = udf(ascii_ignore)  # pyspark.sql.functions include udf()

clean_users = users.withColumn("clean_tweet_text", ascii_udf('tweet_text'))
clean_users.show()
clean_users.count()

+--------------------+-----------+-------------+--------------------+------------------+---------------------+---------------------+--------------------+--------------------+---------------+--------------------+
|                 _id|coordinates|retweet_count|              source|          tweet_ID|tweet_followers_count|tweet_mentioned_count|          tweet_text|                user|      user_name|    clean_tweet_text|
+--------------------+-----------+-------------+--------------------+------------------+---------------------+---------------------+--------------------+--------------------+---------------+--------------------+
|[578ffa8e7eb9513f...|       null|            0|<a href="http://t...|755891629932675072|                  461|                    1|RT @ochocinco: I ...|[[2011-12-27T09:0...|        koteras|RT  ochocinco  I ...|
|[578ffa8f7eb9513f...|       null|            0|<a href="http://t...|755891632759681024|                 4601|                    3|RT @NiallOfficial...

11188

In [68]:
clean_users = clean_users.withColumn("clean_tweet_text",lower(col("clean_tweet_text"))) # pyspark.sql.functions include lower() and col()

In [40]:
# clean_users.select("tweet_text").where(clean_users.tweet_text.contains("http://")).count()

8543

In [69]:
# Perform WordCount on the cleaned tweet texts. (note: this is several lines.)
lines = clean_users.select("clean_tweet_text").rdd.flatMap(lambda x:x)

In [70]:
words = lines.flatMap(lambda line : line.split(" "))

In [ ]:
# For Checking
# yo = words.collect()
# from pyspark.sql.types import *
# lets = spark.createDataFrame(yo, StringType())
# lets.collect()
# # lets.toPandas().to_csv('mycsv.csv')
# lets.where(lets.value == "nigeria").count()

89

In [71]:
tuples = words.map(lambda word: (word , 1))

In [72]:
words.count() # Total words detected

279709

In [73]:
counts = tuples.reduceByKey(lambda a , b: (a + b))

In [74]:
type(counts)

pyspark.rdd.PipelinedRDD

In [75]:
type(counts.collect())

list

In [76]:
counts.take(3)

[('', 69563), ('i', 2039), ('beat', 57)]

In [77]:
counts.count()

18173

In [78]:
# Create the DataFrame of tweet word counts
WordCount = spark.createDataFrame(counts, ["Words", "Count"])
WordCount.sort(desc("Count")).show()
WordCount.count()

+---------+-----+
|    Words|Count|
+---------+-----+
|         |69563|
|        t|10651|
|    https|10075|
|       co| 9543|
|       rt| 5976|
|       to| 5975|
|      you| 4934|
|      the| 4799|
| football| 4424|
|     fifa| 4116|
|     when| 4043|
|     from| 2353|
|        a| 2288|
|       he| 2131|
|        i| 2039|
|    could| 1936|
|     away| 1892|
|      run| 1885|
|       94| 1882|
|throwback| 1844|
+---------+-----+
only showing top 20 rows



18173

In [39]:
# WordCount.where(WordCount.Words == "http").show()

+-----+-----+
|Words|Count|
+-----+-----+
| http|   49|
+-----+-----+



In [79]:
 CountryDF = country_lines.selectExpr("Country as Words")
 CountryDF.show()

+-------------------+
|              Words|
+-------------------+
|        Afghanistan|
|            Albania|
|            Algeria|
|     American Samoa|
|            Andorra|
|             Angola|
|           Anguilla|
|Antigua and Barbuda|
|          Argentina|
|            Armenia|
|              Aruba|
|          Australia|
|            Austria|
|         Azerbaijan|
|            Bahamas|
|            Bahrain|
|         Bangladesh|
|           Barbados|
|            Belarus|
|            Belgium|
+-------------------+
only showing top 20 rows



In [81]:
CountryDF = CountryDF.withColumn("Words",lower(col("Words")))

In [82]:
# Join the country and tweet DataFrames (on the appropriate column)
MyPyaraTable = WordCount.join(CountryDF, on = ["Words"], how = 'inner') # I joined with Country only because Codes did not exist in tweet texts

MyPyaraTable.sort(asc("Words")).collect()
# MyPyaraTable.count()

[Row(Words='albania', Count=2),
 Row(Words='argentina', Count=3),
 Row(Words='australia', Count=2),
 Row(Words='austria', Count=7),
 Row(Words='bahamas', Count=1),
 Row(Words='belgium', Count=1),
 Row(Words='bermuda', Count=1),
 Row(Words='brazil', Count=18),
 Row(Words='canada', Count=12),
 Row(Words='chad', Count=9),
 Row(Words='chile', Count=1),
 Row(Words='colombia', Count=2),
 Row(Words='denmark', Count=1),
 Row(Words='england', Count=43),
 Row(Words='finland', Count=1),
 Row(Words='france', Count=80),
 Row(Words='gambia', Count=1),
 Row(Words='georgia', Count=7),
 Row(Words='germany', Count=25),
 Row(Words='ghana', Count=3),
 Row(Words='greece', Count=1),
 Row(Words='guinea', Count=8),
 Row(Words='hungary', Count=1),
 Row(Words='iceland', Count=5),
 Row(Words='india', Count=4),
 Row(Words='iran', Count=1),
 Row(Words='iraq', Count=6),
 Row(Words='israel', Count=4),
 Row(Words='italy', Count=4),
 Row(Words='jamaica', Count=2),
 Row(Words='japan', Count=8),
 Row(Words='jordan', Cou

In [83]:
MyPyaraTable.select("Words").distinct().count()

56

In [84]:
MyPyaraTable.registerTempTable("PyaraTable") #So Sql Queries can Run

# **Question 1.1:  how many different countries are mentioned in the tweets**

In [85]:
# Question 1: number of distinct countries mentioned
MyPyaraTable.count()

56

In [86]:
# Also By SQL Query
Counting = spark.sql("""Select Count(*) AS Count_Mention From PyaraTable""")
Counting.show()

+-------------+
|Count_Mention|
+-------------+
|           56|
+-------------+



# **Question 1.2:   computes the total number of times any country is mentioned**

In [87]:
# Question 2: number of countries mentioned in tweets.
from pyspark.sql.functions import sum
# MyPyaraTable.printSchema()
MyPyaraTable.groupBy().sum().show()

+----------+
|sum(Count)|
+----------+
|       584|
+----------+



In [88]:
# Also By SQL Query
CountriesMentioned = spark.sql("""Select Sum(Count) AS Sum_Mention From PyaraTable""")
CountriesMentioned.show()

+-----------+
|Sum_Mention|
+-----------+
|        584|
+-----------+



# **Question 1.3:   Your next task is to determine the most popular countries. You can do this by finding the three countries mentioned the most.**

In [89]:
# Table 1: top three countries and their counts.
# from pyspark.sql.functions import desc

# MyPyaraTable.sort(desc("Count")).show(truncate = 3) #Something Cool
# MyPyaraTable.sort(desc("Count")).take(3) #Another Way to accomplish it
MyPyaraTable.sort(desc("Count")).show(3)

+-------+-----+
|  Words|Count|
+-------+-----+
|nigeria|   89|
| france|   80|
| norway|   54|
+-------+-----+
only showing top 3 rows



In [90]:
# Also By SQL Query
DesccendingOrder = spark.sql("""SELECT * FROM PyaraTable ORDER BY Count Desc LIMIT 3;""")
DesccendingOrder.show()

+-------+-----+
|  Words|Count|
+-------+-----+
|nigeria|   89|
| france|   80|
| norway|   54|
+-------+-----+



# **Question 1.4:  you are now interested in how many times specific countries are mentioned. For example, how many times was France mentioned?**

In [91]:
MyPyaraTable.where(MyPyaraTable.Words == 'france').show()

+------+-----+
| Words|Count|
+------+-----+
|france|   80|
+------+-----+



In [92]:
# Also By SQL Query
WhereFrance = spark.sql("""SELECT * FROM PyaraTable WHERE Words = 'france';""")
WhereFrance.show()

+------+-----+
| Words|Count|
+------+-----+
|france|   80|
+------+-----+



# **Question 1.5:   Which country has the most mentions: Kenya, Wales, or Netherlands?**

In [93]:
MyPyaraTable.where( (MyPyaraTable.Words == 'kenya') | (MyPyaraTable.Words == 'wales') | (MyPyaraTable.Words == 'netherlands') ).sort(desc("Count")).show(1)

+-----+-----+
|Words|Count|
+-----+-----+
|wales|   29|
+-----+-----+
only showing top 1 row



In [94]:
# Also By SQL Query
TopAmong3 = spark.sql("""SELECT * FROM PyaraTable WHERE Words = 'kenya' OR Words = 'wales' OR Words = 'netherlands' ORDER BY 2 DESC LIMIT 1 ;""")
TopAmong3.show()

+-----+-----+
|Words|Count|
+-----+-----+
|wales|   29|
+-----+-----+



# **Question 2.6:   Finally, what is the average number of times a country is mentioned?**

In [95]:
# MyPyaraTable.groupBy().avg("Count").show() #Also possible
MyPyaraTable.groupBy().agg(avg("Count").alias("Average Number of Country Mentioned")).show()

+-----------------------------------+
|Average Number of Country Mentioned|
+-----------------------------------+
|                 10.428571428571429|
+-----------------------------------+



In [96]:
# Also By SQL Query
AverageMention = spark.sql("""SELECT AVG(Count) AS Country_Avg_Mention FROM PyaraTable ;""")
AverageMention.show()

+-------------------+
|Country_Avg_Mention|
+-------------------+
| 10.428571428571429|
+-------------------+



In [ ]:
# A good line of code we should know
# df.filter(df.location.contains('google.com'))

In [ ]:
# Table 2: counts for Wales, Iceland, and Japan.


# **Link for Colab Python Notebook: https://colab.research.google.com/drive/1M3JOxXqgILOOyB6t2zFPQVf_kqviTDMY?usp=sharing**